## Understanding WaitGroup

Chúng ta chạy thử đoạn code dưới:

In [1]:
func boring(msg string) {
    for i := 0; ; i++ {
        fmt.Printf("%s %d\n", msg, i)
        time.Sleep(time.Second)
    }
}

func main() {
    go boring("boring!")
    fmt.Println("I'm listening.")
    fmt.Println("You're boring; I'm leaving.")
}

I'm listening.
You're boring; I'm leaving.


Chúng ta thấy rằng func boring không in ra thứ gì, hàm main (cũng là 1 goroutine) in ra 2 dòng như trên code rồi thoát chương trình. Lý do là vì:
- main cũng là 1 goroutine, nó được khởi tạo đầu tiên, các goroutine chỉ được khởi tạo sau main
- main chỉ in ra 2 dòng, quá nhanh, rồi sau đó terminate
- main bị terminate sẽ terminate toàn bộ goroutine khác, lúc này boring goroutine chưa kịp in ra msg gì thì đã bị terminate


Đó là lý do WaitGroup ra đời. WaitGroup sẽ chờ các goroutines thực thi xong. Main goroutine sẽ:
- `Add`: set số lượng các goroutines mà main sẽ chờ
- Mỗi goroutine phải gọi `Done` khi thực thi xong
- `Wait` sẽ block chương trình cho đến khi toàn bộ các goroutines đều gọi `Done`

In [ ]:
var wg sync.WaitGroup

wg.Add(1) // Khai báo số lượng goroutines (gọi là counter) trong Add()
go func() {
    defer wg.Done() // Gọi Done() khi hoàn thành, mỗi lần gọi Done() thì counter ở trên sẽ giảm đi 1
    // Do work
}()

wg.Wait() // Sẽ chờ cho đến khi counter ở trên về 0

## Basic WaitGroup Patterns

### 1. Parallel Task Execution

Giả sử chúng ta có 1 list tasks cần xử lý song song:
- `tasks` là 1 string slice, chứa nhiều task bên trong
- `processTask` sẽ xử lí các task này, hoàn thành trong thời gian random trong 3s
- `parallelTasks` sẽ nhận list task ở trên, và gọi các goroutine `processTask` để xử lí song song

In [2]:
func parallelTasks(tasks []string) {
    var wg sync.WaitGroup
    
    for _, task := range tasks {
        wg.Add(1)
        go func(t string) {
            defer wg.Done()
            processTask(t)
        }(task)
    }
    
    wg.Wait()
    fmt.Println("All tasks completed")
}

func processTask(task string) {
    fmt.Printf("Processing %s\n", task)
    time.Sleep(time.Duration(rand.Intn(3000)) * time.Millisecond)
    fmt.Printf("Completed %s\n", task)
}

func main() {
    tasks := []string{"task1", "task2", "task3", "task4"}
    parallelTasks(tasks)
    fmt.Println("End!")
}

Processing task4
Processing task3
Processing task2
Processing task1
Completed task3
Completed task2
Completed task4
Completed task1
All tasks completed
End!


In [ ]:
for _, task := range tasks {
    wg.Add(1)
    go func(t string) {
        defer wg.Done()
        processTask(t)
    }(task)
}

Chúng ta thấy là trong vòng lặp, với mỗi task sẽ gọi 1 goroutine để xử lí và tương ứng với mỗi task cũng sẽ có 1 WaitGroup

### 2. Fan-Out Pattern
Đây là pattern chia công việc cho nhiều workers làm

In [9]:
package main

import (
	"fmt"
	"math/rand"
	"sync"
	"time"
)

type jobResult struct {
	worker int
	idx    int
	val    int
}

func process(r *rand.Rand, x int) int {
	// simulate variable work time
	time.Sleep(time.Duration(r.Intn(800)+200) * time.Millisecond)
	return x * x
}

func fanOutFanIn(data []int, workers int) []int {
	if workers <= 0 {
		workers = 1
	}
	if workers > len(data) {
		workers = len(data)
	}

	results := make([]int, len(data))
	work := make(chan int)
	out := make(chan jobResult)

	var wg sync.WaitGroup

	wg.Add(workers)
	for w := 1; w <= workers; w++ {
		workerID := w

		// each worker gets its own RNG (important)
		rng := rand.New(rand.NewSource(time.Now().UnixNano() + int64(w)))

		go func(r *rand.Rand) {
			defer wg.Done()
			for idx := range work {
				fmt.Printf("[worker %d] got idx=%d\n", workerID, idx)
				val := process(r, data[idx])
				fmt.Printf("[worker %d] finished idx=%d => %d\n", workerID, idx, val)
				out <- jobResult{worker: workerID, idx: idx, val: val}
			}
		}(rng)
	}

	go func() {
		for i := range data {
			work <- i
		}
		close(work)
		wg.Wait()
		close(out)
	}()

	for r := range out {
		results[r.idx] = r.val
	}

	return results
}

func main() {
	data := []int{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
	numWorkers := 4

	fmt.Println("Input:", data)
	results := fanOutFanIn(data, numWorkers)
	fmt.Println("Output:", results)
}


Input: [0 1 2 3 4 5 6 7 8 9 10]
[worker 1] got idx=0
[worker 4] got idx=1
[worker 2] got idx=2
[worker 3] got idx=3
[worker 2] finished idx=2 => 4
[worker 2] got idx=4
[worker 4] finished idx=1 => 1
[worker 4] got idx=5
[worker 3] finished idx=3 => 9
[worker 3] got idx=6
[worker 1] finished idx=0 => 0
[worker 1] got idx=7
[worker 1] finished idx=7 => 49
[worker 1] got idx=8
[worker 2] finished idx=4 => 16
[worker 2] got idx=9
[worker 4] finished idx=5 => 25
[worker 4] got idx=10
[worker 1] finished idx=8 => 64
[worker 3] finished idx=6 => 36
[worker 2] finished idx=9 => 81
[worker 4] finished idx=10 => 100
Output: [0 1 4 9 16 25 36 49 64 81 100]


#### Chúng ta có:
- `data` có 10 numbers
- 4 workers
- và muốn có results có cùng thứ tự như data

#### 1) `jobResult` struct

In [ ]:
type jobResult struct {
	worker int
	idx    int
	val    int
}

Workers sẽ trả lại kết quả sau khi xử lý xong, nhưng nếu chỉ có `val` thì sẽ mất thông tin nó thuộc về index nào:
- `idx`: index, để biết element nào được xử lý
- `val`: giá trị sau khi xử lí xong
- `worker`: worker nào xử lí (ở đây dùng cho debug)

#### 2) `process()` func

In [ ]:
func process(r *rand.Rand, x int) int {
	time.Sleep(time.Duration(r.Intn(800)+200) * time.Millisecond)
	return x * x
}

- sleep 200-1000ms để mô phỏng thời gian xử lý
- trả về `x*x`

#### 3) `fanOutFanIn(data, workers)` để xử lý song song

In [ ]:
if workers <= 0 { workers = 1 }
if workers > len(data) { workers = len(data) }

Đoạn này để giới hạn số workers

In [ ]:
results := make([]int, len(data))
work := make(chan int)
out := make(chan jobResult)
var wg sync.WaitGroup

- `results`: đây là kết quả sau khi xử lý xong toàn bộ data đầu vào, có độ dài bằng data
- `work`: đây là channel để gửi idx tới các workers
- `out`: đây là channel để các workers gửi kết quả `results` vào jobResult
- `wg`: waitgroup để đảm bảo các workers xử lý xong data

#### 4) Start worker goroutines (fan-out)

In [ ]:
wg.Add(workers)
for w := 1; w <= workers; w++ {
	workerID := w
	rng := rand.New(rand.NewSource(time.Now().UnixNano() + int64(w)))

	go func(r *rand.Rand) {
		defer wg.Done()
		for idx := range work {
			val := process(r, data[idx])
			out <- jobResult{worker: workerID, idx: idx, val: val}
		}
	}(rng)
}


`wg.Add(workers)`: báo WaitGroup là sẽ có N workers, chờ N lần gọi Done()

Mỗi worker sẽ chạy cùng 1 loop:

In [ ]:
for idx := range work {
   ...
}

**Một worker sẽ được assign 1 job như thế nào?**   
- Cái này sẽ cho channel quyết định, khi nhiều workers đang chờ đợi ở range loop như vầy, Go Runtime sẽ tự động gọi 1 worker nào đó để nhận giá trị kế tiếp
- Do đó, 1 worker nào đó sẽ không được chọn jobs, mà được assign job kế tiếp khi available.  Chúng ta sẽ thấy log kiểu như:   
    \[worker 1\] finished idx=4 => 25   
    \[worker 1\] got idx=9  

#### 5) Feed jobs + đóng các channels

In [ ]:
go func() {
	for i := range data {
		work <- i
	}
	close(work)
	wg.Wait()
	close(out)
}()

1. Gửi toàn bộ job index từ 0 đến len(data) - 1 vào work
2. `close(work)` → báo các workers là không còn job
3. `wg.Wait()` → chờ cho tới khi toàn bộ workers thực thi xong
4. `close(out)` → không còn results nào

**Giả sử chỗ này chúng ta không close(work) thì sao?**

- Giữa các goroutines liên lạc với nhau bằng channels, cụ thể ở đây thì goroutine ở step 4 trên và step 5 này communicate thông qua channel work.
- Nếu ở step 5 không close(work) thì loop ở step 4 sẽ không đóng được, và gây ra lỗi deadlock

#### 6) Collect results (fan-in) — single writer

In [ ]:
for r := range out {
	results[r.idx] = r.val
}

Đoạn này đơn giản là ghi giá trị từ channel out vào results, tránh data race vì workers không edit gì results.   
Thực ra, ghi trực tiếp vào results từ các goroutines cũng được, nhưng nó risky hơn

### 3. Batch Processing

In [10]:
package main

import (
	"fmt"
	"math/rand"
	"sync"
	"time"
)

// processBatch simulates doing work for a batch.
// In real life this could be: bulk insert, calling an API for a batch, etc.
func processBatch(batchID int, b []string, r *rand.Rand) {
	fmt.Printf("[batch %d] start: %v\n", batchID, b)

	// simulate variable processing time per item
	for _, item := range b {
		time.Sleep(time.Duration(r.Intn(200)+80) * time.Millisecond)
		fmt.Printf("[batch %d]   processed: %s\n", batchID, item)
	}

	fmt.Printf("[batch %d] done\n", batchID)
}

// processBatches splits items into batches and runs each batch concurrently.
func processBatches(items []string, batchSize int) {
	if batchSize <= 0 {
		batchSize = 1
	}

	var wg sync.WaitGroup

	batchID := 0
	for i := 0; i < len(items); i += batchSize {
		end := i + batchSize
		if end > len(items) {
			end = len(items)
		}

		batch := items[i:end] // slice view into the same underlying array
		batchID++
		id := batchID

		wg.Add(1)
		go func(b []string, batchID int) {
			defer wg.Done()

			// per-goroutine RNG (avoid deprecated rand.Seed + avoid races)
			r := rand.New(rand.NewSource(time.Now().UnixNano() + int64(batchID)))

			processBatch(batchID, b, r)
		}(batch, id)
	}

	wg.Wait()
	fmt.Println("All batches completed")
}

func main() {
	items := []string{
		"A01", "A02", "A03", "A04", "A05",
		"A06", "A07", "A08", "A09", "A10",
		"A11", "A12", "A13",
	}

	batchSize := 4

	fmt.Printf("Items: %v\n", items)
	fmt.Printf("Batch size: %d\n\n", batchSize)

	processBatches(items, batchSize)
}


Items: [A01 A02 A03 A04 A05 A06 A07 A08 A09 A10 A11 A12 A13]
Batch size: 4

[batch 4] start: [A13]
[batch 2] start: [A05 A06 A07 A08]
[batch 3] start: [A09 A10 A11 A12]
[batch 1] start: [A01 A02 A03 A04]
[batch 3]   processed: A09
[batch 2]   processed: A05
[batch 1]   processed: A01
[batch 3]   processed: A10
[batch 4]   processed: A13
[batch 4] done
[batch 2]   processed: A06
[batch 1]   processed: A02
[batch 3]   processed: A11
[batch 2]   processed: A07
[batch 1]   processed: A03
[batch 2]   processed: A08
[batch 2] done
[batch 1]   processed: A04
[batch 1] done
[batch 3]   processed: A12
[batch 3] done
All batches completed


Pattern này thì khác ở trên:
- Ở trên là data có 10 items và có 4 wokers để xử lý song song
- Ở đây thì items có 13 items, và mỗi goroutine sẽ xử lý 1 batch gồm 4 items => do đó tổng cộng cần ceil (tạm dịch là làm tròn lên) của 13/4 là 4 goroutines

#### So sánh

| Concept         | batchSize                  | workers            |
| --------------- | -------------------------- | ------------------ |
| Controls        | items per goroutine        | goroutines running |
| Goroutine count | depends on data size       | fixed              |
| Load balancing  | ❌ none                    | ✅ automatic       |
| Best for        | batch APIs, bulk DB ops    | uneven / slow jobs |
| Memory          | can create many goroutines | bounded            |


### 4. Hybrid pattern

In [1]:
package main

import (
	"fmt"
	"math/rand"
	"sync"
	"time"
)

// ---- simulate batch processing (e.g. bulk DB insert, API call)
func processBatch(workerID int, batchID int, batch []string, r *rand.Rand) {
	fmt.Printf("[worker %d] start batch %d: %v\n", workerID, batchID, batch)

	for _, item := range batch {
		time.Sleep(time.Duration(r.Intn(200)+100) * time.Millisecond)
		fmt.Printf("[worker %d]   processed %s\n", workerID, item)
	}

	fmt.Printf("[worker %d] done batch %d\n", workerID, batchID)
}

// ---- hybrid batching + worker pool
func processItemsHybrid(items []string, batchSize int, workers int) {
	if batchSize <= 0 {
		batchSize = 1
	}
	if workers <= 0 {
		workers = 1
	}

	// 1️⃣ Create batches
	var batches [][]string
	for i := 0; i < len(items); i += batchSize {
		end := i + batchSize
		if end > len(items) {
			end = len(items)
		}
		batches = append(batches, items[i:end])
	}

	// 2️⃣ Channels
	work := make(chan []string)
	var wg sync.WaitGroup

	// 3️⃣ Start worker pool
	wg.Add(workers)
	for w := 1; w <= workers; w++ {
		workerID := w
		rng := rand.New(rand.NewSource(time.Now().UnixNano() + int64(w)))

		go func(r *rand.Rand) {
			defer wg.Done()
			batchID := 0

			for batch := range work {
				batchID++
				processBatch(workerID, batchID, batch, r)
			}
			fmt.Printf("[worker %d] exiting\n", workerID)
		}(rng)
	}

	// 4️⃣ Send batches to workers
	go func() {
		for _, batch := range batches {
			work <- batch
		}
		close(work) // signal: no more batches
	}()

	// 5️⃣ Wait for workers to finish
	wg.Wait()
	fmt.Println("All batches processed")
}

func main() {
	items := []string{
		"A01", "A02", "A03", "A04", "A05",
		"A06", "A07", "A08", "A09", "A10",
		"A11", "A12", "A13",
	}

	batchSize := 3   // 3 items per batch
	workers := 2     // 2 batches processed in parallel

	fmt.Println("Items:", items)
	fmt.Printf("Batch size = %d, Workers = %d\n\n", batchSize, workers)

	processItemsHybrid(items, batchSize, workers)
}


Items: [A01 A02 A03 A04 A05 A06 A07 A08 A09 A10 A11 A12 A13]
Batch size = 3, Workers = 2

[worker 1] start batch 1: [A01 A02 A03]
[worker 2] start batch 1: [A04 A05 A06]
[worker 2]   processed A04
[worker 1]   processed A01
[worker 2]   processed A05
[worker 1]   processed A02
[worker 2]   processed A06
[worker 2] done batch 1
[worker 2] start batch 2: [A07 A08 A09]
[worker 1]   processed A03
[worker 1] done batch 1
[worker 1] start batch 2: [A10 A11 A12]
[worker 2]   processed A07
[worker 1]   processed A10
[worker 2]   processed A08
[worker 2]   processed A09
[worker 2] done batch 2
[worker 2] start batch 3: [A13]
[worker 1]   processed A11
[worker 2]   processed A13
[worker 2] done batch 3
[worker 2] exiting
[worker 1]   processed A12
[worker 1] done batch 2
[worker 1] exiting
All batches processed


## Advanced WaitGroup Patterns (Pending)